# Part I. Pre-Processing the raw csv Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Get list of all raw csv files

In [2]:
# Path to the folder containing the raw csv files
filepath = os.getcwd() + '/event_data'

# For loop to iterate over each file and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    
print(file_path_list)

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/work

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# List that will contain all the data rows from all the raw csv files
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        
        #ignoring the first line of the file, which contains the column names
        next(csvreader)
        
        # extracting each data row one by one and appending to the list        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Number of rows in the newly created csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling using Apache Cassandra 

## The newly created CSV file titled <font color=red>event_datafile_new.csv</font>, located within the current directory, contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the pre-processed data looks like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Making a connection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection for executing queries, creating a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                    WITH REPLICATION = 
                    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:

try :
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### With Apache Cassandra, database tables are modelled keeping in mind the queries we want to run. For this project, we have 3 types of queries. We will create separate tables for each type of query.

#### Sample Query 1 : Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### For this type of query, we will create a table called <font color=blue>**"songsBySessions"**</font> where the PRIMARY KEY is <font color=blue>**(sessionId, itemInSession)**</font>.

In [8]:
query = """CREATE TABLE IF NOT EXISTS songsBySessions \
        (artist text, song text, length float, sessionId int, itemInSession int, \
        PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

                 

In [9]:
#Inserting data to the songsBySessions table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)

    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO songsBySessions (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [10]:
#SELECT statement to verify the data inserted into the table
query = """SELECT artist, song, length from songsBySessions where sessionId = 338 and itemInSession = 4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### For this type of query, we will create a table called <font color=blue>**"songsByUsers"**</font> where the PRIMARY KEY is <font color=blue>**(userId, sessionId, itemInSession)**</font>. Here, we had to include <font color=blue>**itemInSession**</font> in our primary key as we need the songs sorted by itemInSession

In [11]:
#Creating the songsByUsers table
query = """CREATE TABLE IF NOT EXISTS songsByUsers \
        (artist text, song text, itemInSession int, user text, userId int, sessionId int, \
        PRIMARY KEY (userId, sessionId, itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
#Inserting data to the songsByUsers table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)

    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
        query = "INSERT INTO songsByUsers (artist, song, itemInSession, user, userId, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[3]), line[1] + " " + line[4], int(line[10]), int(line[8])))

In [13]:
#SELECT statement to verify the data inserted into the table
query = """SELECT artist, song, user from songsByUsers where userId = 10 and sessionId = 182"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### For this type of query, we will create a table called <font color=blue>**"userBySongs"**</font> where the PRIMARY KEY is <font color=blue>**(song, sessionId)**</font>. Here, we had to include <font color=blue>**sessionId**</font> in our primary key as song names might not be unique.

In [19]:
query = """CREATE TABLE IF NOT EXISTS usersBySongs \
        (user text, song text, sessionId int, \
        PRIMARY KEY (song, sessionId))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)

    next(csvreader) # skip header
    for line in csvreader:
        #print(line)

        query = "INSERT INTO usersBySongs (user, song, sessionId)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[1] + " " + line[4], line[9], int(line[8]) ))

In [22]:

query = """SELECT user from usersBySongs where song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables

In [18]:

drop_query1 = "DROP TABLE IF EXISTS songsBySessions"
drop_query2 = "DROP TABLE IF EXISTS songsByUsers"
drop_query3 = "DROP TABLE IF EXISTS usersBySongs"

try:
    session.execute(drop_query1)
    session.execute(drop_query2)
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [102]:
session.shutdown()
cluster.shutdown()